In [1]:
import pandas as pd
import copy
import random
from load_dataset import load
from classifier import NeuralNetwork, LogisticRegression, SVM
from utils import *
from metrics import *  # include fairness and corresponding derivatives
from expl import get_update_expl
from influence import *
from scipy.optimize import Bounds, minimize

random.seed(1)
np.random.seed(1)
torch.manual_seed(1)

In [2]:
# ignore all the warnings
import warnings
warnings.filterwarnings('ignore') 

**Parameters can be changed:**

**dataset**: german / sqf / adult

**clf_name**: LogisticRegression / SVM / NeuralNetwork

**metric**: The metric for debugging. 0 -> spd, 1 -> tpr parity (equal opportunity), 2 -> predictive parity

**lr**: Learning Rate (can try with different step sizes and choose the best)

**duplicates**: make duplicates of datasets to test runtime vs dataset size with the same feature number and relationships

In [3]:
dataset = 'german'
clf_name = 'LogisticRegression'
metric = 0

**Load Dataset**

In [4]:
X_train, X_test, y_train, y_test = load(dataset)

In [5]:
duplicates = 1
make_duplicates = lambda x, d: pd.concat([x]*d, axis=0).reset_index(drop=True)
X_train = make_duplicates(X_train, duplicates)
X_test = make_duplicates(X_test, duplicates)
y_train = make_duplicates(y_train, duplicates)
y_test = make_duplicates(y_test, duplicates)

In [6]:
X_train_orig = copy.deepcopy(X_train)
X_test_orig = copy.deepcopy(X_test)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

**Loss function**

In [7]:
clf = eval(clf_name)(input_size=X_train.shape[-1])
num_params = len(convert_grad_to_ndarray(list(clf.parameters())))
if isinstance(clf, LogisticRegression):
    loss_func = logistic_loss_torch
elif isinstance(clf, SVM):
    loss_func = svm_loss_torch
elif isinstance(clf, NeuralNetwork):
    loss_func = nn_loss_torch

In [8]:
clf = eval(clf_name)(input_size=X_train.shape[-1])

clf.fit(X_train, y_train)

y_pred_test = clf.predict_proba(X_test)
y_pred_train = clf.predict_proba(X_train)

spd_0 = computeFairness(y_pred_test, X_test_orig, y_test, 0, dataset)
print("Initial statistical parity: ", spd_0)

tpr_parity_0 = computeFairness(y_pred_test, X_test_orig, y_test, 1, dataset)
print("Initial TPR parity: ", tpr_parity_0)

predictive_parity_0 = computeFairness(y_pred_test, X_test_orig, y_test, 2, dataset)
print("Initial predictive parity: ", predictive_parity_0)

accuracy_0 = computeAccuracy(y_test, y_pred_test)
print("Initial accuracy: ", accuracy_0)

Initial statistical parity:  -0.09527580118738121
Initial TPR parity:  -0.07785149359511678
Initial predictive parity:  -0.10136869102808022
Initial accuracy:  0.755


**Select delta fairness function depending on selected metric**

In [9]:
metric_val = [spd_0, tpr_parity_0, predictive_parity_0][metric]
del_F_del_theta = get_del_F_del_theta(clf, X_test_orig, X_test, y_test, dataset, metric)

**Select pattern for update**

In [10]:
X_train_orig.head(5)

,status,duration,credit_hist,credit_amt,savings,employment,install_rate,debtors,residence,property,...,purpose_A42,purpose_A43,purpose_A44,purpose_A45,purpose_A46,purpose_A48,purpose_A49,housing_A151,housing_A152,housing_A153
0,3,1,2,0,4,3,4,0,4,2,...,0,0,0,0,0,0,0,1,0,0
1,3,0,2,1,4,4,4,0,3,1,...,0,0,0,0,0,0,0,0,1,0
2,2,1,2,1,1,2,1,0,1,1,...,0,0,0,0,0,0,0,0,1,0
3,0,0,2,0,2,0,1,0,2,2,...,0,0,0,0,0,0,0,1,0,0
4,0,1,2,1,0,2,2,0,2,3,...,0,1,0,0,0,0,0,0,1,0


In [11]:
# From previous experience [10, 30, 50, 70, 90] would be a good set of options
lr = 50

In [12]:
# A pattern to be updated would be a joint of predicates that describes a subset of data.
# We provide pattern recommendations below for each dataset.
# But it would be interesting to select the pattern yourself to explore which pattern is good to be updated.
# You can select the patterns based on the dataset preview above

if dataset=='german':
    pattern_idx = (X_train_orig.age==1)&(X_train_orig.gender==0)  # recommended for german
elif dataset=='adult':
    pattern_idx = (X_train_orig.gender==0)&(X_train_orig.relationship==0)  # recommended for adult
elif dataset=='sqf':
    pattern_idx = (X_train_orig.cs_descr==1)&(X_train_orig.ht_feet==1)\
                  &(X_train_orig.race==0)&(X_train_orig.sex_F==0)  # recommended for sqf

In [13]:
X_train_updated = get_update_expl(clf, lr, pattern_idx, X_train_orig,
                                  X_train, y_train, del_F_del_theta, loss_func)

100%|██████████| 40/40 [00:09<00:00,  4.06it/s]


**Test effectiveness of the update**

In [14]:
clf = eval(clf_name)(input_size=X_train.shape[-1])

clf.fit(X_train_updated, y_train)

y_pred_test = clf.predict_proba(X_test)
y_pred_train = clf.predict_proba(X_train)

if metric==0:
    spd_0 = computeFairness(y_pred_test, X_test_orig, y_test, 0, dataset)
    print("Updated statistical parity: ", spd_0)
elif metric==1:
    tpr_parity_0 = computeFairness(y_pred_test, X_test_orig, y_test, 1, dataset)
    print("Updated TPR parity: ", tpr_parity_0)
else:
    predictive_parity_0 = computeFairness(y_pred_test, X_test_orig, y_test, 2, dataset)
    print("Updated predictive parity: ", predictive_parity_0)

accuracy_0 = computeAccuracy(y_test, y_pred_test)
print("Updated accuracy: ", accuracy_0)

Updated statistical parity:  -0.018946960102766752
Updated accuracy:  0.775
